# Clustering Consulting Project 

A large technology firm needs your help, they've been hacked! Luckily their forensic engineers have grabbed valuable data about the hacks, including information like session time,locations, wpm typing speed, etc. The forensic engineer relates to you what she has been able to figure out so far, she has been able to grab meta data of each session that the hackers used to connect to their servers. These are the features of the data:

* 'Session_Connection_Time': How long the session lasted in minutes
* 'Bytes Transferred': Number of MB transferred during session
* 'Kali_Trace_Used': Indicates if the hacker was using Kali Linux
* 'Servers_Corrupted': Number of server corrupted during the attack
* 'Pages_Corrupted': Number of pages illegally accessed
* 'Location': Location attack came from (Probably useless because the hackers used VPNs)
* 'WPM_Typing_Speed': Their estimated typing speed based on session logs.


The technology firm has 3 potential hackers that perpetrated the attack. Their certain of the first two hackers but they aren't very sure if the third hacker was involved or not. They have requested your help! Can you help figure out whether or not the third suspect had anything to do with the attacks, or was it just two hackers? It's probably not possible to know for sure, but maybe what you've just learned about Clustering can help!

**One last key fact, the forensic engineer knows that the hackers trade off attacks. Meaning they should each have roughly the same amount of attacks. For example if there were 100 total attacks, then in a 2 hacker situation each should have about 50 hacks, in a three hacker situation each would have about 33 hacks. The engineer believes this is the key element to solving this, but doesn't know how to distinguish this unlabeled data into groups of hackers.**

In [1]:
import findspark
findspark.init('/home/stuthi/prep/spark-2.0.0-bin-hadoop2.7/')

In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('clus').getOrCreate()

In [5]:
df = spark.read.csv("hack_data.csv",header=True,inferSchema=True)

In [6]:
from pyspark.ml.clustering import KMeans

In [7]:
df.head(2)

[Row(Session_Connection_Time=8.0, Bytes Transferred=391.09, Kali_Trace_Used=1, Servers_Corrupted=2.96, Pages_Corrupted=7.0, Location='Slovenia', WPM_Typing_Speed=72.37),
 Row(Session_Connection_Time=20.0, Bytes Transferred=720.99, Kali_Trace_Used=0, Servers_Corrupted=3.04, Pages_Corrupted=9.0, Location='British Virgin Islands', WPM_Typing_Speed=69.08)]

In [8]:
from pyspark.ml.feature import VectorAssembler

In [9]:
assm = VectorAssembler(inputCols=['Session_Connection_Time', 'Bytes Transferred', 'Kali_Trace_Used',
             'Servers_Corrupted', 'Pages_Corrupted','WPM_Typing_Speed'], outputCol='features')

In [10]:
feature_df = assm.transform(df)

In [11]:
feature_df.show(2)

+-----------------------+-----------------+---------------+-----------------+---------------+--------------------+----------------+--------------------+
|Session_Connection_Time|Bytes Transferred|Kali_Trace_Used|Servers_Corrupted|Pages_Corrupted|            Location|WPM_Typing_Speed|            features|
+-----------------------+-----------------+---------------+-----------------+---------------+--------------------+----------------+--------------------+
|                    8.0|           391.09|              1|             2.96|            7.0|            Slovenia|           72.37|[8.0,391.09,1.0,2...|
|                   20.0|           720.99|              0|             3.04|            9.0|British Virgin Is...|           69.08|[20.0,720.99,0.0,...|
+-----------------------+-----------------+---------------+-----------------+---------------+--------------------+----------------+--------------------+
only showing top 2 rows



In [12]:
from pyspark.ml.feature import StandardScaler

In [13]:
scaler = StandardScaler(inputCol='features',outputCol='scaledFeature',withMean=True, withStd=True)

In [14]:
scaled_df = scaler.fit(feature_df).transform(feature_df)

In [15]:
scaled_df.head(2)

[Row(Session_Connection_Time=8.0, Bytes Transferred=391.09, Kali_Trace_Used=1, Servers_Corrupted=2.96, Pages_Corrupted=7.0, Location='Slovenia', WPM_Typing_Speed=72.37, features=DenseVector([8.0, 391.09, 1.0, 2.96, 7.0, 72.37]), scaledFeature=DenseVector([-1.5622, -0.7549, 0.9749, -0.9985, -1.2529, 1.1205])),
 Row(Session_Connection_Time=20.0, Bytes Transferred=720.99, Kali_Trace_Used=0, Servers_Corrupted=3.04, Pages_Corrupted=9.0, Location='British Virgin Islands', WPM_Typing_Speed=69.08, features=DenseVector([20.0, 720.99, 0.0, 3.04, 9.0, 69.08]), scaledFeature=DenseVector([-0.7105, 0.3972, -1.0227, -0.9638, -0.6001, 0.8752]))]

In [17]:
kmeans2 = KMeans(featuresCol='scaledFeature',k=2)
kmeans3 = KMeans(featuresCol='scaledFeature',k=3)

In [18]:
k2model = kmeans2.fit(scaled_df)
k3model = kmeans3.fit(scaled_df)

In [19]:
k2cost = k2model.computeCost(scaled_df)
k3cost = k3model.computeCost(scaled_df)

In [20]:
k2cost

601.7707512676687

In [21]:
k3cost

434.149289871582

In [23]:
for k in range(2,9):
    kmeans = KMeans(featuresCol='scaledFeature',k=k)
    model = kmeans.fit(scaled_df)
    wssse = model.computeCost(scaled_df)
    print("With K={}".format(k))
    print("Within Set Sum of Squared Errors = " + str(wssse))
    print('--'*30)

With K=2
Within Set Sum of Squared Errors = 601.7707512676687
------------------------------------------------------------
With K=3
Within Set Sum of Squared Errors = 434.149289871582
------------------------------------------------------------
With K=4
Within Set Sum of Squared Errors = 267.13361168878964
------------------------------------------------------------
With K=5
Within Set Sum of Squared Errors = 253.1434158580805
------------------------------------------------------------
With K=6
Within Set Sum of Squared Errors = 232.6205419416794
------------------------------------------------------------
With K=7
Within Set Sum of Squared Errors = 213.29483644873758
------------------------------------------------------------
With K=8
Within Set Sum of Squared Errors = 209.41314860027651
------------------------------------------------------------


In [28]:
k3model.transform(scaled_df).groupBy('prediction').count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|         1|   84|
|         2|   83|
|         0|  167|
+----------+-----+



In [29]:
k2model.transform(scaled_df).groupBy('prediction').count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|         1|  167|
|         0|  167|
+----------+-----+

